In [1]:
!pip install fuzzywuzzy==0.18.0
!pip install wget==3.2
!pip install plotly==5.6.0

In [2]:
import wget

wget.download('https://leidata-preview.gleif.org/storage/golden-copy-files/2022/04/01/617937/20220401-0800-gleif-goldencopy-lei2-golden-copy.csv.zip#')
wget.download('https://leidata-preview.gleif.org/storage/golden-copy-files/2022/04/01/617985/20220401-0800-gleif-goldencopy-rr-golden-copy.csv.zip#')

'20220401-0800-gleif-goldencopy-rr-golden-copy.csv.zip'

In [3]:
import pandas as pd
import plotly.express as px
col = ['LEI',
        'Entity.LegalName',
        'Entity.LegalJurisdiction',
        'Entity.LegalForm.EntityLegalFormCode',
        'Entity.EntityStatus',
        'Entity.RegistrationAuthority.RegistrationAuthorityEntityID',
        'Entity.EntityCategory',
        'Registration.RegistrationStatus',
        'Registration.LastUpdateDate' ]

data = pd.read_csv('20220401-0800-gleif-goldencopy-lei2-golden-copy.csv.zip', compression='zip', usecols = col,dtype='unicode')
data_rr = pd.read_csv('20220401-0800-gleif-goldencopy-rr-golden-copy.csv.zip',compression='zip',dtype='unicode')

# Exe 1: Public Limited Companies (PLCs) in Great Britain

In [4]:
#Entity.LegalForm.EntityLegalFormCode = ‘B6ES’
plc_gb = data[data['Entity.LegalForm.EntityLegalFormCode']=='B6ES'].drop_duplicates()
plc_gb = plc_gb.applymap(str)
plc_gb.head()

,LEI,Entity.LegalName,Entity.RegistrationAuthority.RegistrationAuthorityEntityID,Entity.LegalJurisdiction,Entity.EntityCategory,Entity.LegalForm.EntityLegalFormCode,Entity.EntityStatus,Registration.LastUpdateDate,Registration.RegistrationStatus
3626,0TMBS544NMO7GLCE7H90,ABRDN PLC,SC286832,GB,GENERAL,B6ES,ACTIVE,2022-03-28T14:21:02.157Z,ISSUED
4275,2138001123F8QFPOZ919,TANGENT COMMUNICATIONS PLC,03967805,GB,GENERAL,B6ES,ACTIVE,2019-01-15T13:19:01.703Z,LAPSED
4313,21380011BEY3JSQ3AU68,PARSLEY BOX GROUP PLC,SC685656,GB,GENERAL,B6ES,ACTIVE,2022-03-22T04:13:11.243Z,ISSUED
4325,21380011EU95E93AQN91,SULNOX GROUP PLC,08449586,GB,GENERAL,B6ES,ACTIVE,2022-03-22T04:13:43.710Z,ISSUED
4352,21380011L2Y2O7QAFQ89,NORTHERN & SHELL PLC,04086466,GB,GENERAL,B6ES,ACTIVE,2021-05-14T12:18:16.910Z,ISSUED


In [5]:
plc_gb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4114 entries, 3626 to 2105045
Data columns (total 9 columns):
 #   Column                                                      Non-Null Count  Dtype 
---  ------                                                      --------------  ----- 
 0   LEI                                                         4114 non-null   object
 1   Entity.LegalName                                            4114 non-null   object
 2   Entity.RegistrationAuthority.RegistrationAuthorityEntityID  4114 non-null   object
 3   Entity.LegalJurisdiction                                    4114 non-null   object
 4   Entity.EntityCategory                                       4114 non-null   object
 5   Entity.LegalForm.EntityLegalFormCode                        4114 non-null   object
 6   Entity.EntityStatus                                         4114 non-null   object
 7   Registration.LastUpdateDate                                 4114 non-null   object
 8   Reg

# Exe 2: Visualize the EntityStatus and RegistrationStatus from exercise 1

**Entity.EntityStatus:**

ACTIVE - As of the last report or update, the Legal Entity reported that it was legally registered and operating.

INACTIVE - It has been determined that the Entity that was assigned the LEI is no longer legally registered and/or operating, whether as a result of business closure, acquisition by or merger with another (or new) Entity, or determination of illegitimacy.

new NULL - The entity status is not applicable.

**Registration.RegistrationStatus**

PENDING_VALIDATION - An application for an LEI that has been submitted and which is being processed and validated

ISSUED - An LEI Registration that has been validated and issued, and which identifies an Entity that was an operating Legal Entity as of the last update

DUPLICATE - An LEI Registration that has been determined to be a duplicate registration of the same Legal Entity as another LEI Registration; the DUPLICATE status is assigned to the non-surviving registration (i.e. the LEI that should no longer be used)

LAPSED - An LEI registration that has not been renewed by the NextRenewalDate and is not known by public sources to have ceased operation

deprecated MERGED - An LEI registration for an Entity that has been merged into another Legal Entity, such that this Legal Entity no longer exists as an operating Entity

RETIRED - An LEI registration for an Entity that has ceased operation, without having been merged into another Entity

ANNULLED - An LEI registration that was marked as erroneous or invalid after it was issued

CANCELLED - An LEI registration that was abandoned prior to issuance of an LEI
TRANSFERRED - An LEI registration that has been transferred to a different LOU as the managing LOU

PENDING_TRANSFER - An LEI registration that has been requested to be transferred to another LOU. The request is being processed at the sending LOU

PENDING_ARCHIVAL - An LEI registration is about to be transferred to a different LOU, after which its registration status will revert to a non-pending status

In [6]:
fig = px.histogram(data_frame=plc_gb,
                    x='Entity.EntityStatus', 
                    color='Registration.RegistrationStatus',
                    text_auto=True,
                    barmode='stack',
                    histfunc='count')

fig.update_layout(title = 'EntityStatus and RegistrationStatus relationship',
                  xaxis_title="Entity Status",
                  yaxis_title="TotalRegistration Status", 
                  legend_title='Registration Status')
fig.show()

**How do the fields EntityStatus and RegistrationStatus are related to each other?**

* RegistrationStatus of ISSUED (validated and issued registration) and LAPSED (unknown renewal) are labeled ACTIVE EntityStatus.
* RegistrationStatus of RETIRED (ceased operation) is labeled INACTIVE EntityStatus
* RegistrationStatus of DUPLICATED (same legal entity) is labeled NULL Entity Status



In [7]:
#Check overall relationship
data[['Entity.EntityStatus']] = data[['Entity.EntityStatus']].fillna('NULL')
data[['Entity.EntityStatus','Registration.RegistrationStatus']].groupby(['Entity.EntityStatus','Registration.RegistrationStatus']).count()

Empty DataFrame
Columns: []
Index: [(ACTIVE, ISSUED), (ACTIVE, LAPSED), (ACTIVE, PENDING_ARCHIVAL), (ACTIVE, PENDING_TRANSFER), (INACTIVE, RETIRED), (NULL, ANNULLED), (NULL, DUPLICATE), (NULL, ISSUED), (NULL, PENDING_TRANSFER)]

# Exe 3: Find the ultimate parents of all entities from exercise 1

**Relationship.RelationshipType**

IS_DIRECTLY_CONSOLIDATED_BY - The StartNode of the "child" entity has its accounts fully consolidated by the EndNode "parent" entity, in the sense given by the accounting standard(s) specified in RelationshipQualifiers;the EndNode entity is the closest fully consolidating parent to the StartNode entity in any applicable hierarchical ownership structure.

IS_ULTIMATELY_CONSOLIDATED_BY - The StartNode "child" entity has its accounts fully consolidated by the EndNode "parent" entity, in the sense given by the accounting standard(s) specified in the RelationshipQualifiers; the EndNode entity is the most distant fully consolidating parent from the StartNode entity in any applicable hierarchical ownership structure.

IS_INTERNATIONAL_BRANCH_OF - The StartNode is an international branch of the legal entity designated by EndNode (in jurisdiction country of StartNode). The EndNode is the Head Office and SHALL be an LEI.

IS_FUND-MANAGED_BY - The StartNode is a fund managed by a main management entity. The EndNode is legally responsible for the constitution and operation of the fund.

IS_SUBFUND_OF - The StartNode is a sub-fund to an umbrella fund. The EndNode is a legal entity with one or more than one sub-funds/compartments where each sub-fund/compartment has its own investment objectives, separate investment policies and strategies, segregation of assets, separate investors and which has segregated liability between sub-funds/compartments.

IS_FEEDER_TO - The StartNode is a Feeder Fund,that is (almost) exclusively invested in a single other fund. The EndNode is the Master Fund that has identical investment strategies.

In [8]:
#LEI can be found in RR database
all_rr_id = pd.concat([data_rr['Relationship.EndNode.NodeID'], data_rr['Relationship.StartNode.NodeID']]).drop_duplicates().tolist()
plc_gb_filter = plc_gb[plc_gb['LEI'].isin(all_rr_id)]
plc_gb_filter.head()

,LEI,Entity.LegalName,Entity.RegistrationAuthority.RegistrationAuthorityEntityID,Entity.LegalJurisdiction,Entity.EntityCategory,Entity.LegalForm.EntityLegalFormCode,Entity.EntityStatus,Registration.LastUpdateDate,Registration.RegistrationStatus
3626,0TMBS544NMO7GLCE7H90,ABRDN PLC,SC286832,GB,GENERAL,B6ES,ACTIVE,2022-03-28T14:21:02.157Z,ISSUED
4275,2138001123F8QFPOZ919,TANGENT COMMUNICATIONS PLC,03967805,GB,GENERAL,B6ES,ACTIVE,2019-01-15T13:19:01.703Z,LAPSED
4352,21380011L2Y2O7QAFQ89,NORTHERN & SHELL PLC,04086466,GB,GENERAL,B6ES,ACTIVE,2021-05-14T12:18:16.910Z,ISSUED
4470,21380012AJC5OQUSKX05,PARAGON MORTGAGES (NO.15) PLC,06212267,GB,GENERAL,B6ES,ACTIVE,2021-02-28T00:05:11.030Z,LAPSED
4741,21380013V6JI2T3MOG16,HAMPSHIRE TRUST BANK PLC,01311315,GB,GENERAL,B6ES,ACTIVE,2021-10-12T13:22:18.730Z,ISSUED


In [9]:
#LEI can not be found in RR database: no record in level 2 data
print("{} LEIs don't have Relationship records".format(plc_gb.shape[0]-plc_gb_filter.shape[0]))

2869 LEIs don't have Relationship records


In [10]:
#Relationship.RelationshipType ==  ‘IS_ULTIMATELY_CONSOLIDATED_BY’.
data_rr_filter = data_rr.loc[(data_rr['Relationship.RelationshipType']=='IS_ULTIMATELY_CONSOLIDATED_BY')].drop_duplicates()
data_rr_filter = data_rr_filter.applymap(str)

In [11]:
#LEI that can be found in StartNode
in_start_node = pd.merge(plc_gb_filter[['LEI']], data_rr_filter, left_on='LEI', right_on='Relationship.StartNode.NodeID', how='inner')
print("{} LEIs are the StartNode to their ultimate parent, including case StartNodeID == EndNodeID".format(in_start_node.shape[0]))
in_start_node.head()

608 LEIs are the StartNode to their ultimate parent, including case StartNodeID == EndNodeID


,LEI,Relationship.StartNode.NodeID,Relationship.StartNode.NodeIDType,Relationship.EndNode.NodeID,Relationship.EndNode.NodeIDType,Relationship.RelationshipType,Relationship.RelationshipStatus,Relationship.Period.1.startDate,Relationship.Period.1.endDate,Relationship.Period.1.periodType,...,Relationship.Quantifiers.5.QuantifierAmount,Relationship.Quantifiers.5.QuantifierUnits,Registration.InitialRegistrationDate,Registration.LastUpdateDate,Registration.RegistrationStatus,Registration.NextRenewalDate,Registration.ManagingLOU,Registration.ValidationSources,Registration.ValidationDocuments,Registration.ValidationReference
0,21380012AJC5OQUSKX05,21380012AJC5OQUSKX05,LEI,213800S1TDKIB1IUTS72,LEI,IS_ULTIMATELY_CONSOLIDATED_BY,ACTIVE,2016-10-01T00:00:00Z,2017-09-30T00:00:00Z,ACCOUNTING_PERIOD,...,nan,nan,2014-02-24T00:00:00Z,2021-02-28T00:05:11.030Z,LAPSED,2021-02-27T00:00:00Z,213800WAVVOPS85N2205,FULLY_CORROBORATED,ACCOUNTS_FILING,https://beta.companieshouse.gov.uk/company/062...
1,21380013V6JI2T3MOG16,21380013V6JI2T3MOG16,LEI,213800K3UGDI64TUAH80,LEI,IS_ULTIMATELY_CONSOLIDATED_BY,ACTIVE,2020-01-01T00:00:00Z,2020-12-31T00:00:00Z,ACCOUNTING_PERIOD,...,nan,nan,2014-11-27T00:00:00Z,2021-10-12T13:22:18.730Z,PUBLISHED,2022-11-27T00:00:00Z,213800WAVVOPS85N2205,FULLY_CORROBORATED,ACCOUNTS_FILING,https://find-and-update.company-information.se...
2,21380018AJDKNF3A6712,21380018AJDKNF3A6712,LEI,2138002P5RNKC5W2JZ46,LEI,IS_ULTIMATELY_CONSOLIDATED_BY,ACTIVE,2019-03-01T00:00:00Z,2020-02-29T00:00:00Z,ACCOUNTING_PERIOD,...,nan,nan,2013-12-19T00:00:00Z,2021-11-16T13:41:18.493Z,PUBLISHED,2023-01-09T00:00:00Z,213800WAVVOPS85N2205,FULLY_CORROBORATED,ACCOUNTS_FILING,https://www.tescoplc.com/media/755761/tes006_a...
3,2138001GRMZ39JRSE316,2138001GRMZ39JRSE316,LEI,549300O5FACQ67KI6R92,LEI,IS_ULTIMATELY_CONSOLIDATED_BY,ACTIVE,2020-04-01T00:00:00Z,2021-03-31T00:00:00Z,ACCOUNTING_PERIOD,...,nan,nan,2014-02-18T00:00:00Z,2022-03-21T13:28:11.730Z,PUBLISHED,2023-03-16T00:00:00Z,213800WAVVOPS85N2205,FULLY_CORROBORATED,ACCOUNTS_FILING,https://find-and-update.company-information.se...
4,2138001JRCRN7WGSM347,2138001JRCRN7WGSM347,LEI,WXD0EHQRPI7HKN3I5T57,LEI,IS_ULTIMATELY_CONSOLIDATED_BY,ACTIVE,2020-01-01T00:00:00Z,2020-12-31T00:00:00Z,ACCOUNTING_PERIOD,...,nan,nan,2019-05-20T00:00:00Z,2022-03-25T06:16:18.177Z,PUBLISHED,2023-05-20T00:00:00Z,213800WAVVOPS85N2205,FULLY_CORROBORATED,ACCOUNTS_FILING,https://www.ybs.co.uk/pdfs/financial-results/a...


In [12]:
# Series of LEI can not be found in StartNode
not_in_start_node = plc_gb_filter[~plc_gb_filter['LEI'].isin(in_start_node['Relationship.StartNode.NodeID'])]

# LEI is in EndNode: are ultimate parents of their Child Entities
in_end_node = pd.merge(not_in_start_node[['LEI']], data_rr_filter, left_on='LEI', right_on='Relationship.EndNode.NodeID', how='inner')
in_end_node = in_end_node.groupby('LEI').agg({lambda x: ', '.join((set(x)))})
in_end_node.columns = in_end_node.columns.get_level_values(0)
in_end_node = in_end_node.reset_index()
print("{} LEIs are the EndNode, e.i. the ultimate parents of their child entities".format(in_end_node.shape[0]))
in_end_node.head()

559 LEIs are the EndNode, e.i. the ultimate parents of their child entities


,LEI,Relationship.StartNode.NodeID,Relationship.StartNode.NodeIDType,Relationship.EndNode.NodeID,Relationship.EndNode.NodeIDType,Relationship.RelationshipType,Relationship.RelationshipStatus,Relationship.Period.1.startDate,Relationship.Period.1.endDate,Relationship.Period.1.periodType,...,Relationship.Quantifiers.5.QuantifierAmount,Relationship.Quantifiers.5.QuantifierUnits,Registration.InitialRegistrationDate,Registration.LastUpdateDate,Registration.RegistrationStatus,Registration.NextRenewalDate,Registration.ManagingLOU,Registration.ValidationSources,Registration.ValidationDocuments,Registration.ValidationReference
0,0TMBS544NMO7GLCE7H90,"549300JQ4E2V16CGLN33, 2138001I1CXGD29OLB60, 54...",LEI,0TMBS544NMO7GLCE7H90,LEI,IS_ULTIMATELY_CONSOLIDATED_BY,ACTIVE,"2018-10-01T00:00:00Z, 2014-01-21T00:00:00Z, 20...","2020-12-31T00:00:00Z, 2019-09-30T00:00:00Z, 20...","RELATIONSHIP_PERIOD, ACCOUNTING_PERIOD",...,nan,nan,"2017-09-18T00:00:00Z, 2018-10-30T00:00:00Z, 20...","2021-09-27T13:42:22.107Z, 2021-09-27T12:40:07....","LAPSED, PUBLISHED","2023-04-03T00:00:00Z, 2022-11-29T00:00:00Z, 20...","EVK05KS7XY1DEII3R011, 213800WAVVOPS85N2205, 54...","ENTITY_SUPPLIED_ONLY, FULLY_CORROBORATED","OTHER_OFFICIAL_DOCUMENTS, REGULATORY_FILING, A...",https://www.aberdeenstandard.com/docs?editionI...
1,213800167WOVOK5ZC776,213800H4GKEF3CJ8C774,LEI,213800167WOVOK5ZC776,LEI,IS_ULTIMATELY_CONSOLIDATED_BY,ACTIVE,2020-01-01T00:00:00Z,2020-12-31T00:00:00Z,ACCOUNTING_PERIOD,...,nan,nan,2019-09-09T00:00:00Z,2021-10-29T12:44:22.907Z,PUBLISHED,2022-10-28T00:00:00Z,213800WAVVOPS85N2205,FULLY_CORROBORATED,ACCOUNTS_FILING,https://find-and-update.company-information.se...
2,21380016X1OWIRVRSI65,213800HZXY86ZJ7KA323,LEI,21380016X1OWIRVRSI65,LEI,IS_ULTIMATELY_CONSOLIDATED_BY,ACTIVE,2019-10-06T00:00:00Z,2020-10-03T00:00:00Z,ACCOUNTING_PERIOD,...,nan,nan,2017-09-27T00:00:00Z,2021-08-04T09:38:07.120Z,PUBLISHED,2022-09-28T00:00:00Z,213800WAVVOPS85N2205,FULLY_CORROBORATED,ACCOUNTS_FILING,https://find-and-update.company-information.se...
3,2138001AVBSD1HSC6Z10,"2138009C97IBNLCEIP38, 2138006EGNJJ4L6S6987, 21...",LEI,2138001AVBSD1HSC6Z10,LEI,IS_ULTIMATELY_CONSOLIDATED_BY,ACTIVE,"2020-04-01T00:00:00Z, 2002-09-05T18:30:00.001Z...","nan, 2021-03-31T00:00:00Z, 2019-03-31T00:00:00Z","RELATIONSHIP_PERIOD, ACCOUNTING_PERIOD",...,nan,nan,"2021-11-19T10:03:04.127Z, 2016-12-13T00:00:00Z...","2019-11-19T15:46:06.680Z, 2021-11-19T10:03:04....","LAPSED, PUBLISHED","2023-01-24T00:00:00Z, 2020-01-08T00:00:00Z, 20...","213800WAVVOPS85N2205, 335800FVH4MOKZS9VH40","ENTITY_SUPPLIED_ONLY, FULLY_CORROBORATED",ACCOUNTS_FILING,"URL not publicly available, validated from doc..."
4,2138001I27OUBAF22K83,213800EIVRN53COLPP92,LEI,2138001I27OUBAF22K83,LEI,IS_ULTIMATELY_CONSOLIDATED_BY,ACTIVE,2020-01-01T00:00:00Z,2020-12-31T00:00:00Z,ACCOUNTING_PERIOD,...,nan,nan,2021-12-07T00:00:00Z,2021-12-07T07:41:10.633Z,PUBLISHED,2022-12-07T00:00:00Z,213800WAVVOPS85N2205,FULLY_CORROBORATED,ACCOUNTS_FILING,https://find-and-update.company-information.se...


In [13]:
# RR database contains 'Relationship.RelationshipType' other than 'IS_ULTIMATELY_CONSOLIDATED_BY'
anti_data_rr_filter = data_rr.loc[(data_rr['Relationship.RelationshipType']!='IS_ULTIMATELY_CONSOLIDATED_BY')]

#Series of LEI can not be found in Start or EndNode of 'IS_ULTIMATELY_CONSOLIDATED_BY' relationship
not_ultimate_relationship = not_in_start_node[~not_in_start_node['LEI'].isin(in_end_node['LEI'])]

In [14]:
#LEI {other relationship} EndNode == StartNode {Ultimate Parent} EndNode
not_ultimate_rr_in_start_node = pd.merge(not_ultimate_relationship[['LEI']], anti_data_rr_filter, left_on='LEI', right_on='Relationship.StartNode.NodeID', how='inner')

print("{} LEIs are the StartNode in relationships other that ultimare child-parent".format(in_end_node.shape[0]))
not_ultimate_rr_in_start_node.head()

559 LEIs are the StartNode in relationships other that ultimare child-parent


,LEI,Relationship.StartNode.NodeID,Relationship.StartNode.NodeIDType,Relationship.EndNode.NodeID,Relationship.EndNode.NodeIDType,Relationship.RelationshipType,Relationship.RelationshipStatus,Relationship.Period.1.startDate,Relationship.Period.1.endDate,Relationship.Period.1.periodType,...,Relationship.Quantifiers.5.QuantifierAmount,Relationship.Quantifiers.5.QuantifierUnits,Registration.InitialRegistrationDate,Registration.LastUpdateDate,Registration.RegistrationStatus,Registration.NextRenewalDate,Registration.ManagingLOU,Registration.ValidationSources,Registration.ValidationDocuments,Registration.ValidationReference
0,2138009GN3HVIT9T1Z72,2138009GN3HVIT9T1Z72,LEI,391200IHCFFNDBNLG964,LEI,IS_DIRECTLY_CONSOLIDATED_BY,ACTIVE,2019-01-01T00:00:00Z,2019-12-31T00:00:00Z,ACCOUNTING_PERIOD,...,NaN,NaN,2015-02-05T00:00:00Z,2021-12-10T07:08:35.580Z,PUBLISHED,2023-02-05T00:00:00Z,213800WAVVOPS85N2205,FULLY_CORROBORATED,ACCOUNTS_FILING,https://find-and-update.company-information.se...
1,2138009ZH1J2B1MR1D80,2138009ZH1J2B1MR1D80,LEI,549300M46A841BTWC767,LEI,IS_DIRECTLY_CONSOLIDATED_BY,ACTIVE,2015-06-16T00:00:00Z,NaN,ACCOUNTING_PERIOD,...,NaN,NaN,2015-06-16T00:00:00Z,2018-05-04T12:03:10.240Z,PUBLISHED,2019-06-16T00:00:00Z,213800WAVVOPS85N2205,ENTITY_SUPPLIED_ONLY,OTHER_OFFICIAL_DOCUMENTS,NaN
2,213800DL377MH46PDY63,213800DL377MH46PDY63,LEI,213800ZD31YIOEE51B19,LEI,IS_DIRECTLY_CONSOLIDATED_BY,ACTIVE,2019-04-01T00:00:00Z,2020-03-31T00:00:00Z,ACCOUNTING_PERIOD,...,NaN,NaN,2014-01-29T00:00:00Z,2022-02-02T11:17:44.200Z,PUBLISHED,2023-03-02T00:00:00Z,213800WAVVOPS85N2205,FULLY_CORROBORATED,ACCOUNTS_FILING,https://find-and-update.company-information.se...
3,213800ESMPQ4RQ7G8351,213800ESMPQ4RQ7G8351,LEI,213800JKM5UQHFJOTZ25,LEI,IS_DIRECTLY_CONSOLIDATED_BY,ACTIVE,2020-04-01T00:00:00Z,2021-03-31T00:00:00Z,ACCOUNTING_PERIOD,...,NaN,NaN,2014-01-29T00:00:00Z,2022-03-11T13:47:58.980Z,PUBLISHED,2023-03-11T00:00:00Z,213800WAVVOPS85N2205,FULLY_CORROBORATED,ACCOUNTS_FILING,https://find-and-update.company-information.se...
4,213800F7AYNHLL1BPR31,213800F7AYNHLL1BPR31,LEI,549300TJPP85XH21TK11,LEI,IS_DIRECTLY_CONSOLIDATED_BY,ACTIVE,2020-07-01T00:00:00Z,2021-06-30T00:00:00Z,ACCOUNTING_PERIOD,...,NaN,NaN,2018-01-15T00:00:00Z,2022-01-18T09:48:08.110Z,PUBLISHED,2023-01-18T00:00:00Z,213800WAVVOPS85N2205,FULLY_CORROBORATED,ACCOUNTS_FILING,https://find-and-update.company-information.se...


In [15]:
not_ultimate_rr_in_start_node_list = not_ultimate_rr_in_start_node['LEI'].tolist()
not_ultimate_rr_in_start_node_anti = not_ultimate_relationship[~not_ultimate_relationship['LEI'].isin(not_ultimate_rr_in_start_node_list)]

not_ultimate_rr_in_end_node = pd.merge(not_ultimate_rr_in_start_node_anti[['LEI']], anti_data_rr_filter, left_on='LEI', right_on='Relationship.EndNode.NodeID', how='inner')
not_ultimate_rr_in_end_node = not_ultimate_rr_in_end_node.groupby('LEI').agg({lambda x: ', '.join((set(x)))})
not_ultimate_rr_in_end_node.columns = not_ultimate_rr_in_end_node.columns.get_level_values(0)
not_ultimate_rr_in_end_node = not_ultimate_rr_in_end_node.reset_index()
print("{} LEIs are the EndNode in relationships other that ultimare child-parent".format(not_ultimate_rr_in_end_node.shape[0]))
not_ultimate_rr_in_end_node.head()

41 LEIs are the EndNode in relationships other that ultimare child-parent


/Users/uyenle/opt/anaconda3/lib/python3.9/site-packages/pandas/core/groupby/generic.py:303: FutureWarning:

Dropping invalid columns in SeriesGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the aggregating function.



,LEI,Relationship.StartNode.NodeID,Relationship.StartNode.NodeIDType,Relationship.EndNode.NodeID,Relationship.EndNode.NodeIDType,Relationship.RelationshipType,Relationship.RelationshipStatus,Relationship.Period.1.startDate,Relationship.Period.1.periodType,Registration.InitialRegistrationDate,Registration.LastUpdateDate,Registration.RegistrationStatus,Registration.NextRenewalDate,Registration.ManagingLOU,Registration.ValidationSources,Registration.ValidationDocuments
0,2138001123F8QFPOZ919,9845009B3D5759E3D209,LEI,2138001123F8QFPOZ919,LEI,IS_DIRECTLY_CONSOLIDATED_BY,ACTIVE,2019-03-31T00:00:00+00:00,ACCOUNTING_PERIOD,2020-04-14T10:05:41+00:00,2022-02-23T15:28:33+00:00,PUBLISHED,2022-04-14T09:34:10+00:00,529900T8BM49AURSDO55,FULLY_CORROBORATED,ACCOUNTS_FILING
1,21380011L2Y2O7QAFQ89,2138001B486RXXI3FA30,LEI,21380011L2Y2O7QAFQ89,LEI,IS_DIRECTLY_CONSOLIDATED_BY,ACTIVE,2018-05-29T00:00:00Z,ACCOUNTING_PERIOD,2018-05-29T00:00:00Z,2018-05-29T08:34:10.437Z,PUBLISHED,2019-05-29T00:00:00Z,213800WAVVOPS85N2205,FULLY_CORROBORATED,ACCOUNTS_FILING
2,2138002P98CV22GBM646,213800U1HNQMN9S7PQ90,LEI,2138002P98CV22GBM646,LEI,IS_DIRECTLY_CONSOLIDATED_BY,ACTIVE,2020-04-01T00:00:00Z,ACCOUNTING_PERIOD,2017-11-13T00:00:00Z,2021-11-22T11:59:27.800Z,PUBLISHED,2022-11-22T00:00:00Z,213800WAVVOPS85N2205,FULLY_CORROBORATED,ACCOUNTS_FILING
3,2138003SD721TYSR3731,213800YFRLD1UJ1DIB54,LEI,2138003SD721TYSR3731,LEI,IS_DIRECTLY_CONSOLIDATED_BY,ACTIVE,2019-04-01T00:00:00Z,ACCOUNTING_PERIOD,2016-10-10T00:00:00Z,2021-08-13T16:34:56.460Z,PUBLISHED,2022-10-10T00:00:00Z,213800WAVVOPS85N2205,FULLY_CORROBORATED,ACCOUNTS_FILING
4,2138004N53RFLL6JDQ41,"213800DZ5XY8AG5COI19, 21380017NR49F34PWQ94",LEI,2138004N53RFLL6JDQ41,LEI,IS_DIRECTLY_CONSOLIDATED_BY,ACTIVE,2020-01-01T00:00:00Z,ACCOUNTING_PERIOD,"2018-07-12T00:00:00Z, 2020-03-11T00:00:00Z","2022-03-01T16:08:40.823Z, 2021-07-01T14:49:19....",PUBLISHED,"2022-07-15T00:00:00Z, 2023-03-11T00:00:00Z",213800WAVVOPS85N2205,"ENTITY_SUPPLIED_ONLY, FULLY_CORROBORATED","ACCOUNTS_FILING, SUPPORTING_DOCUMENTS"


# Exe 4: Identify potential duplicates in Great Britain

In [16]:
import numpy as np
from itertools import product, combinations_with_replacement
from nltk.metrics import *
from fuzzywuzzy import fuzz

/Users/uyenle/opt/anaconda3/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning:

Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning



In [17]:
test_data= plc_gb[['LEI','Entity.LegalName','Registration.RegistrationStatus']]
test_data.head()

,LEI,Entity.LegalName,Registration.RegistrationStatus
3626,0TMBS544NMO7GLCE7H90,ABRDN PLC,ISSUED
4275,2138001123F8QFPOZ919,TANGENT COMMUNICATIONS PLC,LAPSED
4313,21380011BEY3JSQ3AU68,PARSLEY BOX GROUP PLC,ISSUED
4325,21380011EU95E93AQN91,SULNOX GROUP PLC,ISSUED
4352,21380011L2Y2O7QAFQ89,NORTHERN & SHELL PLC,ISSUED


In [18]:
dup_test_data = test_data[test_data['Registration.RegistrationStatus']=='DUPLICATE']
print("{} Duplicates in database.".format(dup_test_data.shape[0]))
dup_test_data

10 Duplicates in database.


,LEI,Entity.LegalName,Registration.RegistrationStatus
92756,213800GBJ1RNNUKP2182,OROGEN PLC,DUPLICATE
110511,213800JEFWNMHV4VS596,TOPAZ FINANCE PLC,DUPLICATE
117233,213800KJWSAW2912ZP76,MANCHESTER & LONDON INVESTMENT TRUST PLC,DUPLICATE
140006,213800OIN8T68PM7HK77,N4 PHARMA PLC,DUPLICATE
769036,5493001W2FMNF0GV7B53,Renishaw P L C,DUPLICATE
769424,5493001WZIZE0GTD2X45,Tobermore Concrete Products Limited,DUPLICATE
994623,549300HXJ164L1L8IM53,President Energy PLC,DUPLICATE
1067553,549300MSQV80HSATBG53,Vodafone Group Public Limited Company,DUPLICATE
1190418,549300V0DVQGSADLDB82,Bristol Water PLC,DUPLICATE
1729864,8YXIFWJW5NSQU7UG6S41,Ensco Plc,DUPLICATE


### Levenshtein similarity

In [19]:
#Creating all posible pairs of Entity.Name
legal_name = pd.DataFrame(combinations_with_replacement(test_data['LEI'], 2), columns=["LEI.Ori","LEI.Dup"])
legal_name = legal_name[legal_name['LEI.Ori']!=legal_name['LEI.Dup']]

legal_name = pd.merge(legal_name,test_data, left_on='LEI.Ori', right_on='LEI',how='left')
legal_name = pd.merge(legal_name.drop('LEI',axis=1),test_data, left_on='LEI.Dup', right_on='LEI',how='left', suffixes=('.Ori','.Dup'))
legal_name = legal_name.drop('LEI',axis=1)

legal_name = legal_name.reindex(columns=['LEI.Ori', 
                      'Entity.LegalName.Ori',
                      'Registration.RegistrationStatus.Ori',
                      'LEI.Dup', 
                      'Entity.LegalName.Dup',
                      'Registration.RegistrationStatus.Dup'])

In [20]:
#Lev_ratio on token level
legal_name["LevScore"] = legal_name.apply(lambda x: fuzz.token_sort_ratio(x['Entity.LegalName.Ori'].lower(),x['Entity.LegalName.Dup'].lower()), axis=1)

#min_lev = legal_name.groupby('id1').min()
legal_name[legal_name['LevScore']==100]

,LEI.Ori,Entity.LegalName.Ori,Registration.RegistrationStatus.Ori,LEI.Dup,Entity.LegalName.Dup,Registration.RegistrationStatus.Dup,LevScore
1180602,21380048ADXM6Z67CT18,RENISHAW P L C,ISSUED,5493001W2FMNF0GV7B53,Renishaw P L C,DUPLICATE,100
4982142,213800HP3XUS3KTGWT63,D4T4 SOLUTIONS PLC,ISSUED,984500DJF88E9FFCA882,D4T4 SOLUTIONS PLC,ISSUED,100
5123817,213800I841D2RKMFK955,N4 PHARMA PLC,ISSUED,213800OIN8T68PM7HK77,N4 PHARMA PLC,DUPLICATE,100
6045312,213800MA2ZN22I4ITA79,PRESIDENT ENERGY PLC,ISSUED,549300HXJ164L1L8IM53,President Energy PLC,DUPLICATE,100
7254754,213800TB53ELEUKM7Q61,VODAFONE GROUP PUBLIC LIMITED COMPANY,ISSUED,549300MSQV80HSATBG53,Vodafone Group Public Limited Company,DUPLICATE,100
8249712,549300G5FDS8FKPXPZ37,Bristol Water PLC,LAPSED,549300V0DVQGSADLDB82,Bristol Water PLC,DUPLICATE,100
8430614,7N0UJMJLXD0MDT5D5086,Ensco PLC,LAPSED,8YXIFWJW5NSQU7UG6S41,Ensco Plc,DUPLICATE,100


### N-Gram & Cosine Similarity

In [21]:
import re
# N-Grams
def ngrams(string, n=3):
    string = re.sub(r'[,-./]|\s+',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

print(ngrams('TANGENT COMMUNICATIONS PLC'))

['TAN', 'ANG', 'NGE', 'GEN', 'ENT', 'NTC', 'TCO', 'COM', 'OMM', 'MMU', 'MUN', 'UNI', 'NIC', 'ICA', 'CAT', 'ATI', 'TIO', 'ION', 'ONS', 'NSP', 'SPL', 'PLC']


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

text = [x for x in test_data["Entity.LegalName"].str.lower()]
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(text).toarray()

vectors = [t for t in tf_idf_matrix]
out = cosine_similarity(vectors)
# set cosine of themselve = 0 
np.fill_diagonal(out,0)
max_cos = np.amax(out, axis = 1)
max_ind = np.argmax(out, axis = 1)

test_data_cosine = test_data.copy()
#reset index to join on max cosine score
test_data_cosine.index = range(test_data_cosine.shape[0])

test_data_cosine['Cosine'] = max_cos.tolist()
test_data_cosine['Index.Dup'] = max_ind.tolist()

#join original data
cosine_df = pd.merge(test_data_cosine, test_data_cosine.loc[:,['LEI','Entity.LegalName','Registration.RegistrationStatus']], 
                     left_on='Index.Dup', right_index=True, 
                     how = 'left',
                     suffixes=('.Ori', '.Dup'))

#Remove the Ori - Dup reverse relationship
cosine_df['Index.Ori'] = cosine_df.index
### Get Unique combo Index
filter_index = cosine_df[['Index.Ori','Index.Dup']]
filter_index = pd.DataFrame(np.sort(filter_index.values, axis=1), columns=filter_index.columns).drop_duplicates()

cosine_df = pd.merge(cosine_df,filter_index[['Index.Ori']], how = 'inner', on ='Index.Ori').drop_duplicates()
cosine_df.sort_values(by='Cosine',ascending = False) 

#Set Cut-Off Threshold for cosine
cosine_df_out= cosine_df[cosine_df['Cosine']>0.98]
cosine_df_out

,LEI.Ori,Entity.LegalName.Ori,Registration.RegistrationStatus.Ori,Cosine,Index.Dup,LEI.Dup,Entity.LegalName.Dup,Registration.RegistrationStatus.Dup,Index.Ori
456,21380048ADXM6Z67CT18,RENISHAW P L C,ISSUED,1.0,3295,5493001W2FMNF0GV7B53,Renishaw P L C,DUPLICATE,297
1894,213800HP3XUS3KTGWT63,D4T4 SOLUTIONS PLC,ISSUED,1.0,4018,984500DJF88E9FFCA882,D4T4 SOLUTIONS PLC,ISSUED,1475
1931,213800I841D2RKMFK955,N4 PHARMA PLC,ISSUED,1.0,2143,213800OIN8T68PM7HK77,N4 PHARMA PLC,DUPLICATE,1530
2263,213800MA2ZN22I4ITA79,PRESIDENT ENERGY PLC,ISSUED,1.0,3488,549300HXJ164L1L8IM53,President Energy PLC,DUPLICATE,1915
2674,213800TB53ELEUKM7Q61,VODAFONE GROUP PUBLIC LIMITED COMPANY,ISSUED,1.0,3555,549300MSQV80HSATBG53,Vodafone Group Public Limited Company,DUPLICATE,2560
3055,549300G5FDS8FKPXPZ37,Bristol Water PLC,LAPSED,1.0,3661,549300V0DVQGSADLDB82,Bristol Water PLC,DUPLICATE,3464
3190,7N0UJMJLXD0MDT5D5086,Ensco PLC,LAPSED,1.0,3933,8YXIFWJW5NSQU7UG6S41,Ensco Plc,DUPLICATE,3869


In [23]:
#Cannot define duplicate
dup_test_data[~dup_test_data['LEI'].isin(cosine_df_out['LEI.Dup'].tolist())]

,LEI,Entity.LegalName,Registration.RegistrationStatus
92756,213800GBJ1RNNUKP2182,OROGEN PLC,DUPLICATE
110511,213800JEFWNMHV4VS596,TOPAZ FINANCE PLC,DUPLICATE
117233,213800KJWSAW2912ZP76,MANCHESTER & LONDON INVESTMENT TRUST PLC,DUPLICATE
769424,5493001WZIZE0GTD2X45,Tobermore Concrete Products Limited,DUPLICATE


#### Other features to identify duplicate


"Entity.LegalJurisdiction", "Entity.EntityCategory", "Entity.LegalAddress"

"RegistrationAuthorityID"

In [24]:
col_extra = ['LEI',
        'Entity.LegalName',
        'Entity.LegalJurisdiction',
        'Entity.LegalForm.EntityLegalFormCode',
        'Entity.EntityStatus',
        'Entity.RegistrationAuthority.RegistrationAuthorityEntityID',
        'Entity.EntityCategory',
        'Registration.RegistrationStatus',
        'Entity.LegalAddress.FirstAddressLine',
        'Entity.LegalAddress.City',
        'Entity.LegalAddress.Region',
        'Entity.LegalAddress.Country',
        'Entity.HeadquartersAddress.FirstAddressLine',
        'Entity.HeadquartersAddress.City',
        'Entity.HeadquartersAddress.Region',
        'Entity.HeadquartersAddress.Country']

data_extra = pd.read_csv('20220401-0800-gleif-goldencopy-lei2-golden-copy.csv.zip', compression='zip', usecols = col_extra,dtype='unicode')

In [25]:
all_dup = pd.concat([cosine_df_out['LEI.Ori'], cosine_df_out['LEI.Dup']]).drop_duplicates().tolist()
data_extra[data_extra["LEI"].isin(all_dup)].sort_values('Entity.LegalName')

,LEI,Entity.LegalName,Entity.LegalAddress.FirstAddressLine,Entity.LegalAddress.City,Entity.LegalAddress.Region,Entity.LegalAddress.Country,Entity.HeadquartersAddress.FirstAddressLine,Entity.HeadquartersAddress.City,Entity.HeadquartersAddress.Region,Entity.HeadquartersAddress.Country,Entity.RegistrationAuthority.RegistrationAuthorityEntityID,Entity.LegalJurisdiction,Entity.EntityCategory,Entity.LegalForm.EntityLegalFormCode,Entity.EntityStatus,Registration.RegistrationStatus
968020,549300G5FDS8FKPXPZ37,Bristol Water PLC,Bridgwater Road,Bristol,GB-BST,GB,PO Box 218,Bristol,GB-BST,GB,02662226,GB,GENERAL,B6ES,ACTIVE,LAPSED
1190418,549300V0DVQGSADLDB82,Bristol Water PLC,Bridgwater Road,Bristol,GB-ENG,GB,Bridgwater Road,Bristol,GB-ENG,GB,02662226,GB,GENERAL,B6ES,NaN,DUPLICATE
100840,213800HP3XUS3KTGWT63,D4T4 SOLUTIONS PLC,Windmill House,Sunbury-on-Thames,NaN,GB,Windmill House,Sunbury-on-Thames,NaN,GB,01892751,GB,GENERAL,B6ES,ACTIVE,ISSUED
2065311,984500DJF88E9FFCA882,D4T4 SOLUTIONS PLC,"First Floor, RR Tower IV, T.V.K. Industrial Es...",CHENNAI,IN-TN,IN,91-93 WINDMILL ROAD,Sunbury-on-Thames,GB-SRY,GB,F04628,GB,BRANCH,B6ES,ACTIVE,ISSUED
1480367,7N0UJMJLXD0MDT5D5086,Ensco PLC,3rd Floor,London,GB-EAL,GB,3rd Floor,London,GB-EAL,GB,07023598,GB,GENERAL,B6ES,ACTIVE,LAPSED
1729864,8YXIFWJW5NSQU7UG6S41,Ensco Plc,6 Chesterfield Gardens,London,NaN,GB,6 Chesterfield Gardens,London,NaN,GB,NaN,GB,GENERAL,B6ES,NaN,DUPLICATE
103679,213800I841D2RKMFK955,N4 PHARMA PLC,6TH FLOOR,LONDON,GB-LND,GB,6TH FLOOR,LONDON,GB-LND,GB,01435584,GB,GENERAL,B6ES,ACTIVE,ISSUED
140006,213800OIN8T68PM7HK77,N4 PHARMA PLC,60 GRACECHURCH STREET,LONDON,GB-LND,GB,WESTON HOUSE,DERBY,GB-DER,GB,01435584,GB,GENERAL,B6ES,NaN,DUPLICATE
127155,213800MA2ZN22I4ITA79,PRESIDENT ENERGY PLC,CARRWOOD PARK,LEEDS,GB-LDS,GB,CARRWOOD PARK,LEEDS,GB-LDS,GB,05104249,GB,GENERAL,B6ES,ACTIVE,ISSUED
994623,549300HXJ164L1L8IM53,President Energy PLC,11 Hill Street,London,NaN,GB,11 Hill Street,London,NaN,GB,05104249,GB,GENERAL,B6ES,NaN,DUPLICATE



"Entity.LegalJurisdiction"	
"Entity.EntityCategory"
"Entity.LegalAddress"

"RegistrationAuthorityID"